In [1]:
import rdkit
import pandas as pd
reframe=pd.read_csv('../dumps/reframe_candidates.csv', encoding='latin1')

In [13]:
from rdkit import Chem, DataStructs
from molvs import Standardizer 
from rdkit.Chem import AllChem, SaltRemover, QED, rdMolDescriptors
s=Standardizer()
remover=SaltRemover.SaltRemover()
m=[Chem.MolFromSmiles(reframe['SMILES'].iloc[i]) for i in range(len(reframe))]

RDKit ERROR: [18:21:56] Explicit valence for atom # 8 N, 4, is greater than permitted


In [14]:
molnames=reframe['Name']
molnames=[j for (i,j) in zip(m, molnames) if i]
m=[i for i in m if i] #remove failed parse
m=[s.standardize(im) for im in m]
m=[remover.StripMol(im) for im in m]


131


In [17]:
fp_test=[AllChem.GetMorganFingerprintAsBitVect(m[i], 2, useChirality=True) for i in range(len(m))]

In [19]:
def fingerprint_to_np(fp):
    bit_string = fp.ToBitString()
    return np.array([int(char) for char in bit_string], dtype=np.uint8)

In [25]:
np_fingerprints_reframe = np.asarray([fingerprint_to_np(ifp) for im,ifp in enumerate(fp_test)])
np.savez('../dumps/reframe_fp.npz', np_fingerprints_reframe,molnames)

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
